In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


In [2]:
# Training settings
batch_size = 64

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


100%|██████████| 9912422/9912422 [00:01<00:00, 6145234.23it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1943052.05it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2168350.61it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 1397076.03it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [32]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [33]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [34]:
model = Net()

In [35]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [39]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [42]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [43]:
for epoch in range(1, 10):
    train(epoch)
    test()

/var/folders/n_/nbl4pgg93n143h_gp4rbmncr0000gn/T/ipykernel_1884/1084732941.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.226584
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.244754
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.163521
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.265051
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.339294
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.123220
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.163690
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.279873
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.125570
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.085500
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.363342
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.121147
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.064340
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.081807
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.147154
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.195570
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.248832
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.294396
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.065456
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.159828
Train Epoch: 1 [12800/60000 (

/var/folders/n_/nbl4pgg93n143h_gp4rbmncr0000gn/T/ipykernel_1884/320650516.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/Users/jaewoogwak/anaconda3/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0933, Accuracy: 9728/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.056117
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.075015
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.187260
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.038080
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.120233
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.274200
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.036407
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.048785
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.081216
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.088681
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.118024
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.055106
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.179859
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.072445
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.039132
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.138045
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.151465
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.144631
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.159154
Train Epoch: 2 [12